## **Multi agent implement using `LangGraph` on `airflow_etltest` Database**

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

from langchain_huggingface import HuggingFaceEmbeddings

## **Importing `Embedding` and `ChatGroq llm`**

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

llm = ChatGroq(model=os.getenv("QWEN_MODEL"))

## Test llm
llm.invoke("test line")

AIMessage(content='<think>\nOkay, the user sent "test line". That\'s pretty short. I need to figure out what they want. Maybe they\'re just checking if the system is working. Since there\'s no specific question, I should ask for more details. Let me make sure to be polite and helpful. Maybe they need help with something but didn\'t specify. I\'ll respond by asking how I can assist them. Keep it open-ended so they can explain further.\n</think>\n\nIt seems like you might be testing the system or just starting a conversation. How can I assist you today? Feel free to ask any questions or let me know if you need help with something specific! 😊', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 137, 'prompt_tokens': 10, 'total_tokens': 147, 'completion_time': 0.264216737, 'prompt_time': 0.000274665, 'queue_time': 0.049974588, 'total_time': 0.264491402}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_f17c2eb555', 'finish_reason': 'stop', 'logprobs': No

# **Multi Agent Workflow Start**

### **Connect To database using `pands read_sql`**

In [4]:
import mysql.connector
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='airflow_etltest'
)

## **Create sql engine**

In [6]:
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost/airflow_etltest')

## **Testing Query Run On the Database**

In [ ]:
from sqlalchemy import text
import pandas as pd

# Your target account holder name
first_name = "John"

# Updated query (table: Employee, column: first_name)
query = text("SELECT * FROM employee WHERE first_name = :first_name")

# Run the query using SQLAlchemy engine
df = pd.read_sql(query, con=engine, params={"first_name": first_name})

# Extract the first result as string context (optional)
context_str = df.iloc[0].to_string()
context_str


'employee_id                        1\nfirst_name                      John\nlast_name                        Doe\nemail           john.doe@example.com\nphone_number            123-456-7890\nhire_date                 2023-01-15\njob_title          Software Engineer\nsalary                       75000.0'